In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
model_input = tf.keras.Input(shape=(256, 256, 3))
#importing baseline resnet 50 from tensorflow
model = tf.keras.applications.ResNet50(weights=None, include_top=True, input_tensor=model_input,classes = 1000) #

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/train.zip -d /content # to unzip the training file in drive

Streaming output truncated to the last 5000 lines.
  inflating: /content/train/n04579145/n04579145_944.JPEG  
  inflating: /content/train/n04579145/n04579145_95.JPEG  
   creating: /content/train/n04579432/
  inflating: /content/train/n04579432/n04579432_10070.JPEG  
  inflating: /content/train/n04579432/n04579432_10312.JPEG  
  inflating: /content/train/n04579432/n04579432_10843.JPEG  
  inflating: /content/train/n04579432/n04579432_11773.JPEG  
  inflating: /content/train/n04579432/n04579432_12525.JPEG  
  inflating: /content/train/n04579432/n04579432_12929.JPEG  
 extracting: /content/train/n04579432/n04579432_145.JPEG  
  inflating: /content/train/n04579432/n04579432_14806.JPEG  
  inflating: /content/train/n04579432/n04579432_14913.JPEG  
  inflating: /content/train/n04579432/n04579432_15260.JPEG  
  inflating: /content/train/n04579432/n04579432_16138.JPEG  
  inflating: /content/train/n04579432/n04579432_16784.JPEG  
  inflating: /content/train/n04579432/n04579432_17230.JPEG  
  

In [ ]:
seed = 389 #random seed

In [ ]:
#all functions defined according to code given in pytorch
#https://github.com/VIPriors/vipriors-challenges-toolkit/blob/master/image-classification/main.py

def normalize_image(image, mean, std):
    for channel in range(3):
        image[:,:,channel] = (image[:,:,channel] - mean[channel]) / std[channel]
    return image
#used on train data 
def random_crop_normalize(img, random_crop_size,mean,std):
    # Note: image_data_format is 'channel_last'
    #return normalize_image(tf.image.random_crop(value = img, size=random_crop_size, seed=seed),mean,std)
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return normalize_image(img[y:(y+dy), x:(x+dx), :],mean,std)

#used on validation data
def center_crop_normalize(img, crop_size,mean,std):
    height, width = img.shape[0], img.shape[1]
    dy, dx = crop_size
    x = (width - dx + 1) // 2
    y = (height - dy + 1) // 2
    return normalize_image(img[y:(y+dy), x:(x+dx), :],mean,std)

def random_crop_normalize_generator(batches, crop_length,mean,std):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop_normalize(batch_x[i], (crop_length, crop_length), mean, std)
        yield (batch_crops, batch_y)
        
def center_crop_normalize_generator(batches, crop_length,mean,std):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = center_crop_normalize(batch_x[i], (crop_length, crop_length), mean, std)
        yield (batch_crops, batch_y)

In [ ]:
#loading data with some random transformations 
'''extra augmentations rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
'''
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  horizontal_flip=True,
                                  fill_mode='nearest',
                                   validation_split=0.2)
training_set = train_datagen.flow_from_directory('./train',
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',subset='training')

validation_set = train_datagen.flow_from_directory('./train',
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',subset='validation')

Found 40000 images belonging to 1000 classes.
Found 10000 images belonging to 1000 classes.


In [ ]:
#values used as specified in the pytorch code provided
training_set_cropped = random_crop_normalize_generator(training_set, 224, mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
validation_set_cropped = center_crop_normalize_generator(validation_set, 224, mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

In [ ]:
#defining loss function and optimizer along with callback for updating learning rate
#!pip install tensorflow_addons
import tensorflow_addons as tfa
l_rate = 0.001 #0.1 #original lr rate
wd = 0.0001
def scheduler(epoch, lr):
    return l_rate * (0.1 ** (epoch // 30))

callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose = 1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/adv1/model_best.h5",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

#loss = tf.keras.losses.sparse_categorical_crossentropy()
loss = tf.keras.losses.CategoricalCrossentropy()
#add weight decay to SGW
optim = tfa.optimizers.SGDW(learning_rate=l_rate, weight_decay=wd, momentum=0.9, nesterov = False, name = 'SGDW')

In [ ]:
class CustomSaver(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if epoch in [1,30,60,90]:  # or save after some epoch, each k-th epoch etc.
            self.model.save("/content/drive/MyDrive/adv1/model_{}.h5".format(epoch))
saver = CustomSaver()

In [ ]:
#model = tf.keras.models.load_model("/content/drive/MyDrive/adv1/model_60.h5")
model.compile(loss=loss,
              optimizer=optim,
              metrics=['accuracy'])
#[keras.metrics.SparseCategoricalAccuracy(name="acc")]
model.fit(training_set_cropped, validation_data = validation_set_cropped, epochs=10,steps_per_epoch=training_set.samples//32,validation_steps=validation_set.samples //32,verbose=1,callbacks = [callback_lr,checkpoint,saver])
model.save("/content/drive/MyDrive/adv1_final")


Epoch 1: LearningRateScheduler setting learning rate to 0.1.
Epoch 1/90
1250/1250 [==============================] - 454s 351ms/step - loss: 7.0102 - accuracy: 8.7500e-04 - val_loss: 6.9105 - val_accuracy: 0.0017 - lr: 0.1000

Epoch 2: LearningRateScheduler setting learning rate to 0.1.
Epoch 2/90
1250/1250 [==============================] - 431s 345ms/step - loss: 6.8977 - accuracy: 0.0012 - val_loss: 6.8849 - val_accuracy: 0.0018 - lr: 0.1000

Epoch 3: LearningRateScheduler setting learning rate to 0.1.
Epoch 3/90
1250/1250 [==============================] - 421s 337ms/step - loss: 6.8489 - accuracy: 0.0019 - val_loss: 6.8063 - val_accuracy: 0.0032 - lr: 0.1000

Epoch 4: LearningRateScheduler setting learning rate to 0.1.
Epoch 4/90
1250/1250 [==============================] - 401s 321ms/step - loss: 6.7546 - accuracy: 0.0035 - val_loss: 6.7438 - val_accuracy: 0.0036 - lr: 0.1000

Epoch 5: LearningRateScheduler setting learning rate to 0.1.
Epoch 5/90
1250/1250 [====================

## Retrained from checkpoint epoch 60 to get the best val accuracy

In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/adv1/model_60.h5")
model.compile(loss=loss,
              optimizer=optim,
              metrics=['accuracy'])
#[keras.metrics.SparseCategoricalAccuracy(name="acc")]
model.fit(training_set_cropped, validation_data = validation_set_cropped, epochs=10,steps_per_epoch=training_set.samples//32,validation_steps=validation_set.samples //32,verbose=1,callbacks = [checkpoint])
model.save("/content/drive/MyDrive/adv1_final")

Epoch 1/10
1250/1250 [==============================] - 458s 354ms/step - loss: 1.7075 - accuracy: 0.7304 - val_loss: 3.6495 - val_accuracy: 0.2735
Epoch 2/10
1250/1250 [==============================] - 433s 346ms/step - loss: 1.9178 - accuracy: 0.7240 - val_loss: 3.6335 - val_accuracy: 0.2794
Epoch 3/10
1250/1250 [==============================] - 416s 333ms/step - loss: 2.1261 - accuracy: 0.7115 - val_loss: 3.6890 - val_accuracy: 0.2804
Epoch 4/10
1250/1250 [==============================] - 402s 322ms/step - loss: 2.3321 - accuracy: 0.6841 - val_loss: 3.7702 - val_accuracy: 0.2704
Epoch 5/10
1250/1250 [==============================] - 392s 313ms/step - loss: 2.5118 - accuracy: 0.6514 - val_loss: 3.8439 - val_accuracy: 0.2613
Epoch 6/10
1250/1250 [==============================] - 385s 308ms/step - loss: 2.6805 - accuracy: 0.6167 - val_loss: 3.9163 - val_accuracy: 0.2567
Epoch 7/10
1250/1250 [==============================] - 385s 308ms/step - loss: 2.8478 - accuracy: 0.5785 - val_

Best val accuracy of 28% found at epoch 63

In [ ]:
model.compile(loss=loss,
              optimizer=optim,
              metrics=['accuracy'])
#[keras.metrics.SparseCategoricalAccuracy(name="acc")]
model.fit(training_set_cropped, validation_data = validation_set_cropped, epochs=90,steps_per_epoch=training_set.samples//32,validation_steps=validation_set.samples //32,verbose=1,callbacks = [callback_lr,checkpoint])
model.save("/content/drive/MyDrive/adv1")


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/90
1250/1250 [==============================] - 448s 353ms/step - loss: 6.9998 - accuracy: 0.0025 - val_loss: 6.8335 - val_accuracy: 0.0048 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/90
1250/1250 [==============================] - 429s 343ms/step - loss: 6.6695 - accuracy: 0.0060 - val_loss: 6.6414 - val_accuracy: 0.0059 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/90
1250/1250 [==============================] - 419s 335ms/step - loss: 6.4336 - accuracy: 0.0105 - val_loss: 6.4813 - val_accuracy: 0.0096 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.001.
Epoch 4/90
1250/1250 [==============================] - 408s 327ms/step - loss: 6.2286 - accuracy: 0.0162 - val_loss: 6.3400 - val_accuracy: 0.0132 - lr: 0.0010

Epoch 5: LearningRateScheduler setting learning rate to 0.001.
Epoch 5/90
1250/1250 [==============

KeyboardInterrupt: ignored

In [ ]:
model.save("/content/drive/MyDrive/adv")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/adv/assets


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Feb 20 22:42:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    39W / 250W |   8765MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------